Antes de crear el modelo, es necesario preparar los datos, incluyendo la fusión de ambos datasets.

In [1]:
import pandas as pd

# Cargar los datasets
insurance_data = pd.read_csv('insurance_data.csv')
vehicle_info = pd.read_csv('vehicle_info.csv')
test_data = pd.read_csv('test.csv')

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

# Fusionar datasets
merged_data = pd.merge(insurance_data, vehicle_info, on='VEHICLE_ID')

# One-Hot Encoding para la variable categórica "TYPE_VEHICLE"
merged_data = pd.get_dummies(merged_data, columns=['TYPE_VEHICLE'], drop_first=True)

# Seleccionar características relevantes
features = merged_data[['CUSTOMER_SENIORITY', 'SEX', 'INSR_TYPE', 'INSURED_VALUE', 'PREMIUM', 'USAGE', 'PROD_YEAR', 'SEATS_NUM', 'CARRYING_CAPACITY', 'CCM_TON'] + [col for col in merged_data.columns if 'TYPE_VEHICLE_' in col]]
target = merged_data['CLAIM_PAID'].notnull().astype(int)  # 1 si hay reclamo, 0 si no

# Codificar variables categóricas
label_encoders = {}
for column in ['SEX', 'INSR_TYPE', 'USAGE']:
    le = LabelEncoder()
    features[column] = le.fit_transform(features[column])
    label_encoders[column] = le

# Imputar valores faltantes
imputer = SimpleImputer(strategy='mean')
features = pd.DataFrame(imputer.fit_transform(features), columns=features.columns)

C:\Users\juant\AppData\Local\Temp\ipykernel_14132\2092013742.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[column] = le.fit_transform(features[column])
C:\Users\juant\AppData\Local\Temp\ipykernel_14132\2092013742.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[column] = le.fit_transform(features[column])
C:\Users\juant\AppData\Local\Temp\ipykernel_14132\2092013742.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

Dividimos los datos en conjuntos de entrenamiento y prueba.

In [3]:
from sklearn.model_selection import train_test_split

# Divide el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

#### Asignación de resultados

- X_train y y_train: Contendrán los datos de características y objetivos, respectivamente, que se usarán para entrenar el modelo.
- X_test y y_test: Contendrán los datos de características y objetivos, respectivamente, que se usarán para evaluar el rendimiento del modelo después del entrenamiento.

#### Parámetros de train_test_split

- features: Conjunto de datos que contiene las características o variables independientes que se usarán para entrenar el modelo.
- target: Conjunto de datos que contiene la variable objetivo o dependiente que el modelo intentará predecir.
- test_size=0.2: Especifica la proporción del conjunto de datos que se debe usar como conjunto de prueba. En este caso, el 20% de los datos se utilizarán para la prueba y el 80% restante para el entrenamiento.
- random_state=42: Establece una semilla para el generador de números aleatorios, lo que garantiza que la división de los datos sea reproducible.

# Selección del Modelo

#### Posibilidades:

1) Regresión Logística: Simple y fácil de interpretar, útil para clasificación binaria.
2) Random Forest (conjunto  de Árboles de Decisión): Capturan relaciones no lineales y son robustos a datos ruidosos.

1) Regresión Logística

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

# Crear y entrenar el modelo
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)

# Realizar predicciones
y_pred_logistic = logistic_model.predict(X_test)

# Evaluar el modelo
print("Regresión Logística")
print(classification_report(y_test, y_pred_logistic))
print("ROC AUC Score:", roc_auc_score(y_test, logistic_model.predict_proba(X_test)[:, 1]))

Regresión Logística
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     74631
           1       0.28      0.01      0.02      6659

    accuracy                           0.92     81290
   macro avg       0.60      0.50      0.49     81290
weighted avg       0.87      0.92      0.88     81290

ROC AUC Score: 0.7259874662832552


La clase 0 representa pólizas que no han hecho un reclamo y la clase 1 representa pólizas que sí han hecho un reclamo. Explicación métricas:

1. Precision = Verdaderos Positivos (VP) / (Verdaderos Positivos (VP) + Falsos Positivos (FP))
- Clase 0: 0.92 (92%): De todas las pólizas que el modelo predijo como sin reclamo (clase 0), el 92% realmente no hicieron un reclamo.
- Clase 1: 0.28 (28%): De todas las pólizas que el modelo predijo como con reclamo (clase 1), solo el 28% realmente hicieron un reclamo.

Interpretación: El modelo es bastante preciso para identificar pólizas sin reclamo (clase 0), pero es menos preciso para identificar aquellas con reclamo (clase 1).

2. Recall = Verdaderos Positivos (VP) / (Verdaderos Positivos (VP) + Falsos Negativos (FP))
- Clase 0: 1.00 (100%): El modelo identificó correctamente el 100% de todas las pólizas que realmente no hicieron un reclamo.
- Clase 1: 0.01 (1%): El modelo identificó correctamente solo el 1% de todas las pólizas que realmente hicieron un reclamo.

Interpretación: El modelo es excelente para detectar pólizas sin reclamo, pero casi no detecta las que realmente hacen un reclamo.

3. F1-Score: media armónica entre la precisión y el recall

4. Accuracy: el 92% de las predicciones totales del modelo fueron correctas.

Interpretación: aunque el valor de exactitud parece alto, es importante recordar que esta métrica puede ser engañosa cuando hay un desbalance en las clases, como parece ser el caso aquí.

5. ROC AUC Score: esta métrica mide la capacidad del modelo para distinguir entre las clases 0 y 1. Un valor de 0.5 indica que el modelo no tiene capacidad de discriminación (equivalente a lanzar una moneda), mientras que un valor de 1.0 indica una discriminación perfecta.

Interpretación: Un ROC AUC de 0.726 indica que el modelo tiene una capacidad razonable para distinguir entre las clases, pero no excelente. A pesar de esto, el modelo parece estar favoreciendo en gran medida la clase 0.



2) Random Forest

In [5]:
from sklearn.ensemble import RandomForestClassifier

# Crear y entrenar el modelo
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Realizar predicciones
y_pred_rf = rf_model.predict(X_test)

# Evaluar el modelo
print("Random Forest")
print(classification_report(y_test, y_pred_rf))
print("ROC AUC Score:", roc_auc_score(y_test, rf_model.predict_proba(X_test)[:, 1]))

Random Forest
              precision    recall  f1-score   support

           0       0.92      0.98      0.95     74631
           1       0.27      0.08      0.13      6659

    accuracy                           0.91     81290
   macro avg       0.60      0.53      0.54     81290
weighted avg       0.87      0.91      0.88     81290

ROC AUC Score: 0.7381735911118705


Interpretación: aunque este modelo es mejor que el anterior, todavía sigue presentando dificultades para detectar las pólizas que realmente hacen un reclamo.

#### Sobremuestreo (Oversampling)

Usamos la técnica SMOTE (Synthetic Minority Oversampling Technique) para generar ejemplos sintéticos de la clase minoritaria.

In [6]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Sobremuestreo usando SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Entrenar el modelo con los datos sobremuestreados
rf_model_smote = RandomForestClassifier(random_state=42)
rf_model_smote.fit(X_resampled, y_resampled)

# Predecir y evaluar
y_pred_smote = rf_model_smote.predict(X_test)
print(classification_report(y_test, y_pred_smote))
print("ROC AUC Score:", roc_auc_score(y_test, rf_model_smote.predict_proba(X_test)[:, 1]))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94     74631
           1       0.24      0.17      0.20      6659

    accuracy                           0.89     81290
   macro avg       0.58      0.56      0.57     81290
weighted avg       0.87      0.89      0.88     81290

ROC AUC Score: 0.7306324621266379


#### Submuestreo (Undersampling)

Alternativamente, podemos reducir la cantidad de ejemplos de la clase mayoritaria.


In [7]:
from imblearn.under_sampling import RandomUnderSampler

# Submuestreo usando RandomUnderSampler
undersampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X_train, y_train)

# Entrenar el modelo con los datos submuestreados
rf_model_undersample = RandomForestClassifier(random_state=42)
rf_model_undersample.fit(X_resampled, y_resampled)

# Predecir y evaluar
y_pred_undersample = rf_model_undersample.predict(X_test)
print(classification_report(y_test, y_pred_undersample))
print("ROC AUC Score:", roc_auc_score(y_test, rf_model_undersample.predict_proba(X_test)[:, 1]))

              precision    recall  f1-score   support

           0       0.97      0.65      0.78     74631
           1       0.16      0.75      0.26      6659

    accuracy                           0.66     81290
   macro avg       0.56      0.70      0.52     81290
weighted avg       0.90      0.66      0.73     81290

ROC AUC Score: 0.7640219131769996


#### Ajuste del Umbral de Clasificación

Por defecto, el modelo de Random Forest clasifica las instancias como positivas (1, pólizas con reclamo) si la probabilidad estimada es mayor que 0.5. Sin embargo, como tus clases están desbalanceadas, podrías cambiar este umbral para favorecer la detección de las pólizas que harán un reclamo.

In [8]:
# Entrenar el modelo Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Obtener las probabilidades predichas
y_prob = rf_model.predict_proba(X_test)[:, 1]  # Probabilidades para la clase 1

# Ajustar el umbral (por ejemplo, 0.3)
threshold = 0.3
y_pred_adjusted = (y_prob >= threshold).astype(int)

# Evaluar el nuevo modelo con el umbral ajustado
print(classification_report(y_test, y_pred_adjusted))
print("ROC AUC Score:", roc_auc_score(y_test, y_prob))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93     74631
           1       0.23      0.22      0.22      6659

    accuracy                           0.88     81290
   macro avg       0.58      0.58      0.58     81290
weighted avg       0.87      0.88      0.87     81290

ROC AUC Score: 0.7381735911118705


#### Combinación de submuestreo y ajuste de umbral

In [9]:
# Aplicar submuestreo (undersampling) a los datos de entrenamiento
undersampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X_train, y_train)

# Entrenar el modelo Random Forest con los datos submuestreados
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_resampled, y_resampled)

# Obtener las probabilidades predichas para el conjunto de prueba
y_prob = rf_model.predict_proba(X_test)[:, 1]  # Probabilidades para la clase 1 (reclamos)

# Ajustar el umbral (por ejemplo, 0.3)
threshold = 0.3
y_pred_adjusted = (y_prob >= threshold).astype(int)

# Evaluar el modelo con el umbral ajustado
print("Classification Report:")
print(classification_report(y_test, y_pred_adjusted))

print("ROC AUC Score:", roc_auc_score(y_test, y_prob))

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.48      0.65     74631
           1       0.13      0.90      0.23      6659

    accuracy                           0.52     81290
   macro avg       0.56      0.69      0.44     81290
weighted avg       0.91      0.52      0.61     81290

ROC AUC Score: 0.7640219131769996


#### Conclusión:

Mi mejor algoritmo es Random Forest con Undersampling (ROC AUC Score: 0.7640).
En este caso, solo el 16% de las pólizas predichas como con reclamo fueron correctas. Por otro lado, el modelo identificó correctamente el 75% de las pólizas que hicieron un reclamo.
Es decir, identifica pólizas que hicieron reclamos exageradamente, pero esto hace que la precisión de captarlas sea mucho más alta.

#### Predicción en el Conjunto de Test

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Cargar datos
insurance_data = pd.read_csv('insurance_data.csv')
vehicle_info = pd.read_csv('vehicle_info.csv')

# Fusionar datasets
merged_data = pd.merge(insurance_data, vehicle_info, on='VEHICLE_ID')

# Seleccionar características y objetivo
features = ['CUSTOMER_SENIORITY', 'SEX', 'INSR_TYPE', 'INSURED_VALUE', 'PREMIUM', 'PROD_YEAR', 'SEATS_NUM', 'CARRYING_CAPACITY', 'CCM_TON']
categorical_features = ['SEX', 'INSR_TYPE']
X = merged_data[features]
y = merged_data['CLAIM_PAID'].notnull().astype(int)  # 1 si hay reclamo, 0 si no

# Dividir datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Configurar ColumnTransformer para preprocesar datos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), ['CUSTOMER_SENIORITY', 'INSURED_VALUE', 'PREMIUM', 'PROD_YEAR', 'SEATS_NUM', 'CARRYING_CAPACITY', 'CCM_TON']),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ]
)

# Crear pipeline de preprocesamiento y clasificación
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Submuestreo usando RandomUnderSampler
undersampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X_train, y_train)

# Ajustar el modelo usando los datos submuestreados
pipeline.fit(X_resampled, y_resampled)

# Realizar predicciones con el conjunto de prueba
test_predictions = pipeline.predict(X_test)  # Usar pipeline directamente para hacer predicciones

# Evaluar el modelo
print(classification_report(y_test, test_predictions))
print("ROC AUC Score:", roc_auc_score(y_test, pipeline.predict_proba(X_test)[:, 1]))

# Si necesitas guardar las predicciones
test_data = pd.DataFrame(X_test)  # Asegúrate de incluir las características que necesitas
test_data['PREDICTED_CLAIM'] = test_predictions
test_data.to_csv('test_predictions.csv', index=False)

              precision    recall  f1-score   support

           0       0.97      0.65      0.78    111950
           1       0.16      0.75      0.26      9985

    accuracy                           0.66    121935
   macro avg       0.56      0.70      0.52    121935
weighted avg       0.90      0.66      0.73    121935

ROC AUC Score: 0.7643659043724138
